<a href="https://colab.research.google.com/github/bernhardtandy/ProjectsMLAI/blob/main/HW6_AndyBernhardt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projects in Machine Learning and AI Homework 6**
## *Reinforcement Learning*
##### **Andy Bernhardt**
##### **bernha@rpi.edu**

---
## **Task 1: Formulating Text Generation as a Markov Decision Process (MDP)**

The goal of the task of text generation is to produce a sequence of words, given a prompt (or no prompt), that is close in quality to text created by humans. Ways to evaluate this may involve measuring coherence, grammaticality/syntactic correctness, semantic meaning, as well as other characteristics of well-formed texts. 

Currently, the most popular approaches to text generation involve training large transformer-based models on the task of causal language modeling with large unsupervised corpora. Given a large number of sequences of texts, these autoregressive models learn to use context from previous tokens in a given sequence to predict a probability distribution over the next token in the same sequence. Then, at test time, the model, starting with the prompt, can sample from these distributions at each step to produce an output sequence. A popular example of this is the GPT-3 model.

This problem could additionally be formulated in the reinforcement learning paradigm as a Markov Decision Process (MDP). For simplicity, we will consider the problem of generating a single English sentence without any prompt; however, we note that this can be extended to longer sequences, with prompts, and for any target languages. We have the following MDP setup for this problem:
- State Space $(S)$: Any finite sequence of English words and punctuation $w = (w_i)$, $i = 0, 1, 2, ... n$, including the empty sequence $()$. There is one initial state $w = ()$. If $w_n \in \{., ?, !\}$, then the state is a terminal state.
- Action Space $(A)$: Any English word or punctuation $v$ from a vocabulary set $V$
- Transitions $(T: S \times A \implies \mathbb{R})$: Given a state $w$ and action $v$, the next state $w'$ is $(w_i + v)$ with probability 1 (deterministic transitions).
- Rewards $(R: S \times A \implies \mathbb{R})$: Given a state $w$ and action $v$, the reward $r$ is a measure of the quality of the next state $w'$. This reward could have a value for every sequence $w'$ (if we are able to find a way to evaluate incomplete sentences), or it could have a value of 0 for every sequence that is not a terminal state (only evaluate a sentence once it has been completed). This measure of quality can be formulated in many ways, including but not limited to:
  - Using a combination of existing NLP metrics for evaluating coherence, grammaticality, meaning, and other desirable characteristics of a text sequence, and giving a positive reward for well-formed sequences, and a negative reward for not well-formed sequences
  - Comparing the sequence to one or more reference sequences from human-created corpora, and giving a positive reward if the generated sequence is close to the reference sequences, and a negative reward if not (for example, BLEU score)

Given this MDP setup, we can run many episodes of this MDP to obtain $(w, v, w', r)$ tuples, and apply Q-learning or deep Q-learning to learn a policy $\pi(v|w)$. Although the optimal policy may be deterministic in this MDP setting, we can choose a less optimal stochastic policy (but still hopefully good) in order to enforce variation in generating new texts. For example, after learning the Q-values $Q(w, v)$: at each state $w_i$, we can choose the top-$k$ words $v_j$, $j = 1, 2, ... k$, that have the highest $Q(w_i, v_j)$ and let $\pi(v_j|w_i)$ be proportional to $\frac{Q(w_i, v_j)}{\sum_{m = 1}^kQ(w_i, v_m)}$.

---
## **Task 2: Reinforcement Learning in Healthcare**

According to Yu et al. [1], reinforcement learning has been applied in many healthcare-related application domains, including "dynamic
treatment regimes in chronic disease or critical care, automated
medical diagnosis, and other general domains such as health
resources allocation and scheduling, optimal process control,
drug discovery and development, as well as health management" (Yu et al. 6). In this section, we will explain the problem of developing dynamic treatment regimes for chronic diseases, and take a look at an open-source project that has addressed this problem.

First, we note that the goal of this task is, for any patients suffering from chronic diseases, to create a dynamic treatment regime (DTR), that can both adapt to "varying clinical states" and "improve the long-term benefits of patients" (7). This involves, at any given time, using a current state of a patient, which can include their "health status" and "prior treatment history" to determine an optimal treatment decision, which can include "treatment type, drug dosage, or reexamination timing" (7). As a result, we see that the problem, formulated in this way, fits into the reinforcement learning paradigm. The states are the patient states at a given time, the actions are the treatment decisions, and what is observed, i.e., the  next state and reward, is a function of the patient state after the treatment decision has taken place. According to Yu et al., applying reinforcement learning to solve this problem has benefits, including that learned policies lead to "time-dependent decisions on the best treatment for each patient at each decision time" and "account for heterogeneity across patients" (7). For chronic diseases, such as cancer, anemia, diabetes, HIV, and mental illnesses, long-term treatment consists of sequences of "medical intervention[s] that must take into account the changing
health status of a patient and adverse effects occurring from
previous treatment"; as a result, for each patient, we have an RL episode consisting of (state, action, next state, and reward) tuples (7). Since 2009, researchers have been applying RL to create dynamic treatment regimes for cancer treatment, with Zhao et al. [2] initially applying a model-free temporal difference method. 

Next, we look at an recent open-source project by Cho et al. [3] that addresses this problem. Initially, the authors note that there are some challenges in the optimal dynamic treatment regime RL problem. These include that "failure time is often censored" and that "treatment timing may not be fixed, may even depend on a patient's status or previous treatment history, and could have association with the failure time" (Cho et al. 2). The authors use a random survival forest, which is an ensemble method for analyzing censored survival data, to "develop a general dynamic treatment regime estimator for censored time-to-failure outcomes", which "maximizes either the mean survival time or the
survival probability at a certain time" (3). Their setup is as follows: Given $n$ independent patients with up to $K$ visits, at time $k$, each patient can receive a treatment if they have survived to time $k$. At each stage, for each patient, they observe historical information, time between stages, treatment, and also the result: failure, advancement, or censoring. Given a dynamic treatment regime $\pi$, the authors would like to estimate the distribution of failure times of patients, and additionally learn an optimal treatment regime $\pi^*$. Similarly to in Q-learning, the authors use a backward recursion/DP algorithm. First, they estimate the "conditional survival distribution" $S_k$ at each time $k$ using the historical information and treatments (6). Next, they find the action that optimizes the next survival distribution. Then, they use the time between stages to "augment the estimated optimal curves" (7). The authors show the theoretical results of the convergence of their algorithm, and show empirical results using a simulation. Additionally, they show the application of their method to a "cardiovascular disease cohort study" (4). In this application for the Atherosclerosis Risk in Communities (ARIC) study, the authors note that there is much censored data. They show that their proposed algorithm for determining an optimal dynamic treatment regime leads to higher estimated mean survival times and higher estimated six-year survival probabilities compared to other previous methods.

[1] Yu, C., Liu, J., & Nemati, S.. (2019). Reinforcement Learning in Healthcare: A Survey. https://arxiv.org/abs/1908.08796.

[2] Y. Zhao, M. R. Kosorok, and D. Zeng, “Reinforcement learning design
for cancer clinical trials,” Statistics in Medicine, vol. 28, no. 26, pp.
3294–3315, 2009. https://pubmed.ncbi.nlm.nih.gov/19750510/.

[3] Cho, H., Holloway, S., Couper, D., & Kosorok, M.. (2020). Multi-stage optimal dynamic treatment regimes for survival outcomes with dependent censoring. https://arxiv.org/abs/2012.03294, https://github.com/Hunyong/survQlearn.

---
## **Task 3: Q-Learning for Tic-Tac-Toe**

In this section, we implement the game of tic-tac-toe, as well as an agent for playing tic-tac-toe, and train two agents to learn the Q function using epsilon-greedy Q-learning over a large number of training episodes, during which the agents play tic-tac-toe against one another. At times, we referred to this useful article, https://towardsdatascience.com/reinforcement-learning-implement-tictactoe-189582bea542, though the way we train the agents and provide the rewards is different. We evaluate the performance of the trained agents by playing a few games of tic-tac-toe against each agent. Since the game is relatively simple, we show that the agents, following their optimal policy, will draw the game against each other, and when playing against untrained agents, will perform very well. 

In [ ]:
# Import requirements
import numpy as np

In [ ]:
# Define the Tic-Tac-Toe environment as a class
class TicTacToe():

  # There are nine total actions, and at each state, only some of these may be valid
  # The board will be represented by a 3 x 3 numpy array
  def __init__(self):
    self.actions = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
    self.current_state = None

  # At the game start, the board is empty (initial state)
  def begin_episode(self):
    self.current_state = np.zeros((3, 3))

  # Given an action and the agent number, the state is updated accordingly
  def transition(self, action, agent_number):
    if (agent_number == 0):
      self.current_state[action[0]][action[1]] = 1
    elif (agent_number == 1):
      self.current_state[action[0]][action[1]] = 2

  # Returns the current state of the board
  def get_current_state(self):
    return self.current_state

  # Returns a string representation of the current state of the board
  def get_current_state_hash(self):
    return str(self.current_state.reshape(9))
  
  # Get the valid actions given the current state: this includes any coordinates that
  # still have no 'x' or 'o'
  def get_valid_actions(self):
    indices = np.where(self.current_state == 0)
    return [(a, b) for (a, b) in zip(indices[0], indices[1])]

  # This function checks if the game has ended. If the game has not ended, returns -1.0
  # If the game has ended in agent 0 winning, return 0.0. If the game has ended in agent
  # 1 winning, return 1.0. If the game has ended in a draw, return 2.0
  def check_end_game(self):
    for i in range(3):
      row_i = self.current_state[i]
      column_i = self.current_state[:, i]
      if ((np.min(row_i) == np.max(row_i)) and np.min(row_i) != 0):
        return np.min(row_i) - 1
      elif ((np.min(column_i) == np.max(column_i)) and np.min(column_i) != 0):
        return np.min(column_i) - 1
    if ((self.current_state[0][0] == self.current_state[1][1]) and (self.current_state[1][1] == self.current_state[2][2]) and (self.current_state[2][2] != 0)):
        return self.current_state[0][0] - 1
    elif ((self.current_state[0][2] == self.current_state[1][1]) and (self.current_state[1][1] == self.current_state[2][0]) and (self.current_state[2][0] != 0)):
        return self.current_state[0][2] - 1
    elif (np.min(self.current_state) != 0):
      return 2.0
    return -1.0

  # Old rewards - revised below
  def reward_old(self, agent_number):
    state_result = self.check_end_game()
    if (state_result == -1.0 or state_result == 2.0):
      return 0.0
    elif (state_result == agent_number):
      return 1.0
    else:
      return -1.0

  # This function checks the current state of the game and gives a reward based on the state and agent number
  def reward(self, agent_number):
    state_result = self.check_end_game()
    if (state_result == -1.0):
      return 0.0
    elif (state_result == 2.0):
      return 1.0
    elif (state_result == agent_number):
      return 10.0
    else:
      return -10.0

  # Displays the Tic-Tac-Toe board, for visualization
  def display(self):
    print("_______")
    for i in range(3):
      line = "|"
      for j in range(3):
        if (self.current_state[i][j] == 0):
          line += "-|" 
        elif (self.current_state[i][j] == 1):
          line += "o|"
        else:
          line += "x|"
      print(line)
    print(u"\u203E\u203E\u203E\u203E\u203E\u203E\u203E")

In [ ]:
# Initialize a Tic-Tac-Toe environment class
tic_tac_toe = TicTacToe()

In [ ]:
# Begin an episode and display the board
tic_tac_toe.begin_episode()
tic_tac_toe.display()
tic_tac_toe.check_end_game()
# -1.0 indicates that the game has not yet ended

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾


-1.0

In [ ]:
# Place an 'o' at (0, 0) and display again
tic_tac_toe.transition((0, 0), 0)
tic_tac_toe.display()
tic_tac_toe.check_end_game()
# -1.0 indicates that the game has not yet ended

_______
|o|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾


-1.0

In [ ]:
# Show that the current set of valid actions is all actions except for (0, 0)
tic_tac_toe.get_valid_actions()

[(0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [ ]:
# Place an 'o' at (1, 1) and display again
tic_tac_toe.transition((1, 1), 0)
tic_tac_toe.display()
tic_tac_toe.check_end_game()
# -1.0 indicates that the game has not yet ended

_______
|o|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾


-1.0

In [ ]:
# Place an 'o' at (2, 2) and display again
tic_tac_toe.transition((2, 2), 0)
tic_tac_toe.display()
tic_tac_toe.check_end_game()
# 0.0 indicates that agent 0 has won the game

_______
|o|-|-|
|-|o|-|
|-|-|o|
‾‾‾‾‾‾‾


0.0

In [ ]:
# Define the Tic-Tac-Toe agent as a class
class TicTacToeAgent():

  # The agent has an agent number, either 0 or 1, which represents if they are the
  # 'o' player or the 'x' player. Alpha, epsilon, and gamma are hyperparameters in the
  # epsilon-greedy Q-learning algorithm, which are tuned during the training
  def __init__(self, agent_number):
    self.agent_number = agent_number
    self.alpha = 0.3
    self.epsilon = 0.5
    self.gamma = 0.95
    self.q = {}

  # This function takes the Tic-Tac-Toe environment state and selects an action
  # using an epsilon-greedy approach. If a uniform sample <= epsilon, then a
  # random valid action is chosen; otherwise, the action is chosen with the maximum
  # Q-value for the current state 
  def select_action(self, tic_tac_toe):
    sample = np.random.uniform(0, 1)
    current_action = None
    current_state = tic_tac_toe.get_current_state_hash()
    valid_actions = tic_tac_toe.get_valid_actions()
    if (current_state not in self.q.keys()):
        self.q[current_state] = {}
        for action in valid_actions:
          self.q[current_state][action] = 0.0

    if (sample <= self.epsilon):
      index = np.random.choice(len(valid_actions))
      current_action = valid_actions[index]

    else:
      best_q_value = -100
      best_action = None
      for k, v in self.q[current_state].items():
        if (v > best_q_value):
          best_q_value = v
          best_action = k 
      current_action = best_action

    return current_state, current_action

  # Given a current state, current action, reward, and new tic-tac-toe environment state,
  # this function updates the agent's Q-table by the Q-learning algorithm update rule  
  def update_q(self, current_state, current_action, reward, tic_tac_toe):
    next_state = tic_tac_toe.get_current_state_hash()
    next_actions = tic_tac_toe.get_valid_actions()
    max_q = -100
    if (next_state not in self.q.keys()):
      max_q = 0.0
    else:
      for action in next_actions:
        if (self.q[next_state][action] > max_q):
          max_q = self.q[next_state][action]
    self.q[current_state][current_action] = self.q[current_state][current_action] + self.alpha * (reward + self.gamma*max_q - self.q[current_state][current_action])

  # This function sets an agent's epsilon for epsilon-greedy action selection
  def set_epsilon(self, epsilon):
    self.epsilon = epsilon

  # This function sets an agent's learning rate alpha
  def set_alpha(self, alpha):
    self.alpha = alpha

  # This function returns the agent's full Q-table 
  def get_q(self):
    return self.q

  # This function returns the agent's Q-values for a given state
  def get_q_s(self, s):
    return self.q[s]
  
  # This function returns the agent's Q-value for a given state/action pair
  def get_q_s_a(self, s, a):
    return self.q[s][a]
  
  # This function returns the value function for the agent's optimal policy
  def get_v_s(self, s):
    return max(self.q[s].values())

  # This function returns the agent's optimal policy
  def get_pi_s(self, s):
    return max(self.q[s], key=self.q[s].get)

In [ ]:
# First attempt at implementing an episode of Q-learning
# The problem is that the rewards are only given for each agent
# directly after that agent moves: therefore, no negative rewards for losing are ever given,
# and the next states are never in the Q-table...
def run_episode_old(disp=True):
  tic_tac_toe.begin_episode()
  if (disp):
    tic_tac_toe.display()

  while(True):
    current_state, a0_action = agent_0.select_action(tic_tac_toe)
    tic_tac_toe.transition(a0_action, 0)
    if (disp):
      tic_tac_toe.display()
    reward = tic_tac_toe.reward(0)
    agent_0.update_q(current_state, a0_action, reward, tic_tac_toe)
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      return check

    current_state, a1_action = agent_1.select_action(tic_tac_toe)
    tic_tac_toe.transition(a1_action, 1)
    if (disp):
      tic_tac_toe.display()
    reward = tic_tac_toe.reward(1)
    agent_1.update_q(current_state, a1_action, reward, tic_tac_toe)
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      return check

In [ ]:
# Second attempt at implementing an episode of Q-learning
# The problem is that for some of the Q updates, the next states are never in the Q-table,
# since the agent_0's Q-table only has states with an even number of filled spots,
# and the agent_1's Q-table only has states with an odd number of filled spots
def run_episode_old_2(epsilon, alpha, disp=True):
  tic_tac_toe.begin_episode()
  agent_0.set_epsilon(epsilon)
  agent_1.set_epsilon(epsilon)
  agent_0.set_alpha(alpha)
  agent_1.set_alpha(alpha)
  if (disp):
    tic_tac_toe.display()

  i = 0
  while(True):
    current_state_0, a0_action = agent_0.select_action(tic_tac_toe)
    tic_tac_toe.transition(a0_action, 0)
    if (disp):
      tic_tac_toe.display()
    reward_0 = tic_tac_toe.reward(0)
    agent_0.update_q(current_state_0, a0_action, reward_0, tic_tac_toe)
    if (i != 0):
      reward_1 = tic_tac_toe.reward(1)
      agent_1.update_q(current_state_1, a1_action, reward_1, tic_tac_toe)
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      reward_1 = tic_tac_toe.reward(1)
      agent_1.update_q(current_state_1, a1_action, reward_1, tic_tac_toe)
      return check

    current_state_1, a1_action = agent_1.select_action(tic_tac_toe)
    tic_tac_toe.transition(a1_action, 1)
    if (disp):
      tic_tac_toe.display()
    reward_0 = tic_tac_toe.reward(0)
    reward_1 = tic_tac_toe.reward(1)
    agent_0.update_q(current_state_0, a0_action, reward_0, tic_tac_toe)
    agent_1.update_q(current_state_1, a1_action, reward_1, tic_tac_toe)
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      reward_0 = tic_tac_toe.reward(0)
      agent_0.update_q(current_state_0, a0_action, reward_0, tic_tac_toe)
      return check
    i += 1

In [ ]:
# Final version of implementing an episode of Q-learning
# First, the agent's epsilon and alpha are set to input parameters; these are decaying throughout training
# Next, the game is initialized. Then, while the game has not yet ended, the agents play the game,
# and rewards full turns after an agent has acted, or if the game has ended
def run_episode(epsilon, alpha, agent_0, agent_1, disp=True):
  tic_tac_toe.begin_episode()
  agent_0.set_epsilon(epsilon)
  agent_1.set_epsilon(epsilon)
  agent_0.set_alpha(alpha)
  agent_1.set_alpha(alpha)
  if (disp):
    tic_tac_toe.display()

  total_reward_0 = 0.0
  total_reward_1 = 0.0
  i = 0
  while(True):
    current_state_0, a0_action = agent_0.select_action(tic_tac_toe)
    tic_tac_toe.transition(a0_action, 0)
    if (disp):
      tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()
    if (i != 0):
      reward_1 = tic_tac_toe.reward(1)
      total_reward_1 += reward_1
      agent_1.update_q(current_state_1, a1_action, reward_1, tic_tac_toe)
    if (check != -1.0):
      reward_0 = tic_tac_toe.reward(0)
      total_reward_0 += reward_0
      agent_0.update_q(current_state_0, a0_action, reward_0, tic_tac_toe)
      reward_1 = tic_tac_toe.reward(1)
      total_reward_1 += reward_1
      agent_1.update_q(current_state_1, a1_action, reward_1, tic_tac_toe)
      return check, agent_0, agent_1, total_reward_0, total_reward_1

    current_state_1, a1_action = agent_1.select_action(tic_tac_toe)
    tic_tac_toe.transition(a1_action, 1)
    if (disp):
      tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()
    reward_0 = tic_tac_toe.reward(0)
    total_reward_0 += reward_0
    agent_0.update_q(current_state_0, a0_action, reward_0, tic_tac_toe)   
    if (check != -1.0):
      reward_0 = tic_tac_toe.reward(0)
      total_reward_0 += reward_0
      agent_0.update_q(current_state_0, a0_action, reward_0, tic_tac_toe)
      reward_1 = tic_tac_toe.reward(1)
      total_reward_1 += reward_1
      agent_1.update_q(current_state_1, a1_action, reward_1, tic_tac_toe)
      return check, agent_0, agent_1, total_reward_0, total_reward_1
    i += 1

In [ ]:
# Runs the training loop for Q-learning of Tic-Tac-Toe
# The starting epsilon is 1, and the starting learning rate is 0.7;
# both decay at a rate of 0.9999 each episode
def run_training(num_episodes, agent_0, agent_1, disp):
  results = {"Agent 0 Wins": 0, "Agent 1 Wins": 0, "Draws": 0}
  epsilon_0 = 1
  alpha_0 = 0.7
  rewards_dict = {}
  for i in range(num_episodes):
    epsilon = epsilon_0 * pow(0.9999, i)
    alpha = alpha_0 * pow(0.9999, i)
    result, agent_0, agent_1, reward_0, reward_1 = run_episode(epsilon, alpha, agent_0, agent_1, disp[i])
    rewards_dict[i] = [reward_0, reward_1]
    if (disp[i] or i % 50 == 0):
      print(f"Episode {i+1}: Epsilon: {epsilon}, Alpha: {alpha}, Result: {result}")
    if (result == 0):
      results["Agent 0 Wins"] += 1
    if (result == 1):
      results["Agent 1 Wins"] += 1
    if (result == 2):
      results["Draws"] += 1
  return results, rewards_dict

In [ ]:
# Initialize a Tic-Tac-Toe environment and two Tic-Tac-Toe agents
tic_tac_toe = TicTacToe()
agent_0 = TicTacToeAgent(0)
agent_1 = TicTacToeAgent(1)

In [ ]:
# Run the training loop for 10000 episodes, and show the games that are played in the last 5 episodes
num_episodes = 10000
disp = [True] * 5 + [False] * (num_episodes - 10)  + [True] * 5
results, rewards_dict = run_training(num_episodes, agent_0, agent_1, disp)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|-|o|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|-|o|
|x|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|-|o|
|x|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|-|o|
|x|-|x|
‾‾‾‾‾‾‾
_______
|o|-|o|
|-|-|o|
|x|-|x|
‾‾‾‾‾‾‾
_______
|o|-|o|
|-|-|o|
|x|x|x|
‾‾‾‾‾‾‾
Episode 1: Epsilon: 1.0, Alpha: 0.7, Result: 1.0
_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|o|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|o|
|-|-|x|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|o|
|-|-|x|
|-|o|-|
‾‾‾‾‾‾‾
_______
|-|-|o|
|-|-|x|
|x|o|-|
‾‾‾‾‾‾‾
_______
|-|-|o|
|-|-|x|
|x|o|o|
‾‾‾‾‾‾‾
_______
|-|-|o|
|x|-|x|
|x|o|o|
‾‾‾‾‾‾‾
_______
|-|o|o|
|x|-|x|
|x|o|o|
‾‾‾‾‾‾‾
_______
|-|o|o|
|x|x|x|
|x|o|o|
‾‾‾‾‾‾‾
Episode 2: Epsilon: 0.9999, Alpha: 0.6999299999999999, Result: 1.0
_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|-|-|
|-|-|o|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|-|-|
|-|x|o|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|-|-|
|-|x|o|
‾‾‾‾‾‾‾
_______
|o|x|-|
|-|-|-|
|-|x|o|
‾‾‾‾‾‾‾
_______
|o|x|-|
|-|-|o|
|-|x|o|
‾‾‾‾‾‾‾
____

In [ ]:
# Print the results of the training
print(results)
# We see that during the 10000 episodes of training, agent 0 won 5417 times, agent 1 won 2930 times, and there were 1653 draws

{'Agent 0 Wins': 5417, 'Agent 1 Wins': 2930, 'Draws': 1653}


In [ ]:
# Get the Q-function for agent 0
q_0 = agent_0.get_q()

In [ ]:
# Show the Q-values for the initial state
q_0['[0. 0. 0. 0. 0. 0. 0. 0. 0.]']
# By the Q-learning, the agent optimally plays in the center on the first turn. This is OK, as playing in the center does guarantee a win or a draw

{(0, 0): 4.590322168242853,
 (0, 1): 4.73489891841905,
 (0, 2): 4.243834047131108,
 (1, 0): 4.837559943164142,
 (1, 1): 5.68253239710333,
 (1, 2): 4.707856698509073,
 (2, 0): 3.5046314344660194,
 (2, 1): 4.2116896215444735,
 (2, 2): 4.294378527558366}

In [ ]:
# Get the Q-function for agent 1
q_1 = agent_1.get_q()

In [ ]:
# Show the Q-values for the state
'''
_______
|-|-|o|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
'''
q_1['[0. 0. 1. 0. 0. 0. 0. 0. 0.]']
# The agent optimally plays 'x' in the center position
# If the agent plays at (1, 0), (1, 2), or (2, 0), they will guaranteed lose against an optimal agent, and so the Q values are negative
# I think some of the other moves are also guaranteed losses...

{(0, 0): 2.733477372685688,
 (0, 1): 1.5319063497373118,
 (1, 0): -2.344159100957398,
 (1, 1): 5.168734482806634,
 (1, 2): -0.763959966270976,
 (2, 0): -4.22302278058896,
 (2, 1): 1.4403168027168995,
 (2, 2): 3.128553076910481}

In [ ]:
# Get agent_0's value for the start state
agent_0.get_v_s('[0. 0. 0. 0. 0. 0. 0. 0. 0.]')

5.68253239710333

In [ ]:
# Get agent_0's value for a guaranteed win state
agent_0.get_v_s('[0. 2. 0. 0. 1. 0. 0. 0. 0.]')

8.729339535324513

In [ ]:
# Get agent_0's policy for the start state
agent_0.get_pi_s('[0. 0. 0. 0. 0. 0. 0. 0. 0.]')

(1, 1)

In [ ]:
# Given epsilon, this function runs an episode of the game for testing between two agents
def run_episode_test(epsilon, agent_0, agent_1):
  agent_0.set_epsilon(epsilon)
  agent_1.set_epsilon(epsilon)
  tic_tac_toe.begin_episode()
  tic_tac_toe.display()

  i = 0
  while(True):
    current_state_0, a0_action = agent_0.select_action(tic_tac_toe)
    tic_tac_toe.transition(a0_action, 0)
    tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      return check

    current_state_1, a1_action = agent_1.select_action(tic_tac_toe)
    tic_tac_toe.transition(a1_action, 1)   
    tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()   
    if (check != -1.0):
      return check
    i += 1

In [ ]:
# When both agents are playing based on their learned optimal policies, they draw
run_episode_test(0.0, agent_0, agent_1)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|x|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|o|x|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|o|x|
|-|o|-|
|-|x|-|
‾‾‾‾‾‾‾
_______
|-|o|x|
|-|o|-|
|-|x|o|
‾‾‾‾‾‾‾
_______
|x|o|x|
|-|o|-|
|-|x|o|
‾‾‾‾‾‾‾
_______
|x|o|x|
|-|o|o|
|-|x|o|
‾‾‾‾‾‾‾
_______
|x|o|x|
|x|o|o|
|-|x|o|
‾‾‾‾‾‾‾
_______
|x|o|x|
|x|o|o|
|o|x|o|
‾‾‾‾‾‾‾


2.0

In [ ]:
# Define a function for playing Tic-Tac-Toe as agent 0 against a provided agent 1
def play_tic_tac_toe_0(epsilon, agent_1):
  agent_1.set_epsilon(epsilon)
  tic_tac_toe.begin_episode()
  tic_tac_toe.display()

  i = 0
  while(True):
    user_input = input('Enter your action (x, y): ')
    a0_action = (int(user_input[1]), int(user_input[4]))
    tic_tac_toe.transition(a0_action, 0)
    tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      return check

    current_state_1, a1_action = agent_1.select_action(tic_tac_toe)
    tic_tac_toe.transition(a1_action, 1)   
    tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()   
    if (check != -1.0):
      return check
    i += 1

In [ ]:
# Define a function for playing Tic-Tac-Toe as agent 1 against a provided agent 0
def play_tic_tac_toe_1(epsilon, agent_0):
  agent_0.set_epsilon(epsilon)
  tic_tac_toe.begin_episode()
  tic_tac_toe.display()

  i = 0
  while(True):
    current_state_0, a0_action = agent_0.select_action(tic_tac_toe)
    tic_tac_toe.transition(a0_action, 0)
    tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      return check

    
    user_input = input('Enter your action (x, y): ')
    a1_action = (int(user_input[1]), int(user_input[4]))
    tic_tac_toe.transition(a1_action, 1)   
    tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()   
    if (check != -1.0):
      return check
    i += 1

In [ ]:
# The next three cells show that I cannot beat the agent when it chooses actions based on its learned optimal policy
play_tic_tac_toe_0(0.0, agent_1)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 0)
_______
|o|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 2)
_______
|o|-|o|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 1)
_______
|o|x|o|
|-|x|-|
|-|o|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|x|x|-|
|-|o|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (1, 2)
_______
|o|x|o|
|x|x|o|
|-|o|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|x|x|o|
|-|o|x|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 0)
_______
|o|x|o|
|x|x|o|
|o|o|x|
‾‾‾‾‾‾‾


2.0

In [ ]:
play_tic_tac_toe_0(0.0, agent_1)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 0)
_______
|o|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 2)
_______
|o|-|-|
|-|x|-|
|-|-|o|
‾‾‾‾‾‾‾
_______
|o|-|-|
|x|x|-|
|-|-|o|
‾‾‾‾‾‾‾
Enter your action (x, y): (1, 2)
_______
|o|-|-|
|x|x|o|
|-|-|o|
‾‾‾‾‾‾‾
_______
|o|-|x|
|x|x|o|
|-|-|o|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 0)
_______
|o|-|x|
|x|x|o|
|o|-|o|
‾‾‾‾‾‾‾
_______
|o|-|x|
|x|x|o|
|o|x|o|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 1)
_______
|o|o|x|
|x|x|o|
|o|x|o|
‾‾‾‾‾‾‾


2.0

In [ ]:
play_tic_tac_toe_1(0.0, agent_0)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 0)
_______
|x|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|x|o|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 1)
_______
|x|o|-|
|-|o|-|
|-|x|-|
‾‾‾‾‾‾‾
_______
|x|o|-|
|o|o|-|
|-|x|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (1, 2)
_______
|x|o|-|
|o|o|x|
|-|x|-|
‾‾‾‾‾‾‾
_______
|x|o|-|
|o|o|x|
|o|x|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 2)
_______
|x|o|x|
|o|o|x|
|o|x|-|
‾‾‾‾‾‾‾
_______
|x|o|x|
|o|o|x|
|o|x|o|
‾‾‾‾‾‾‾


2.0

In [ ]:
# If I give the agent a guaranteed winning position, it will win
play_tic_tac_toe_0(0.0, agent_1)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 0)
_______
|o|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 2)
_______
|o|-|o|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 2)
_______
|o|x|o|
|-|x|-|
|-|-|o|
‾‾‾‾‾‾‾
_______
|o|x|o|
|-|x|-|
|-|x|o|
‾‾‾‾‾‾‾


1.0

In [ ]:
play_tic_tac_toe_1(0.0, agent_0)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 0)
_______
|x|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|x|o|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 0)
_______
|x|o|-|
|-|o|-|
|x|-|-|
‾‾‾‾‾‾‾
_______
|x|o|-|
|-|o|-|
|x|o|-|
‾‾‾‾‾‾‾


0.0

In [ ]:
play_tic_tac_toe_1(0.0, agent_0)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (1, 0)
_______
|-|-|-|
|x|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|x|o|-|
|-|o|-|
‾‾‾‾‾‾‾
Enter your action (x, y): (0, 1)
_______
|-|x|-|
|x|o|-|
|-|o|-|
‾‾‾‾‾‾‾
_______
|-|x|-|
|x|o|-|
|-|o|o|
‾‾‾‾‾‾‾
Enter your action (x, y): (2, 0)
_______
|-|x|-|
|x|o|-|
|x|o|o|
‾‾‾‾‾‾‾
_______
|o|x|-|
|x|o|-|
|x|o|o|
‾‾‾‾‾‾‾


0.0

In [ ]:
# This function runs an episode between two provided agents, agent_0 and agent_1,
# with an additional display flag
def run_game(agent_0, agent_1, disp=True):
  tic_tac_toe.begin_episode()
  if (disp):
    tic_tac_toe.display()

  i = 0
  while(True):
    current_state_0, a0_action = agent_0.select_action(tic_tac_toe)
    tic_tac_toe.transition(a0_action, 0)
    if (disp):
      tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      return check

    current_state_1, a1_action = agent_1.select_action(tic_tac_toe)
    tic_tac_toe.transition(a1_action, 1)
    if (disp):
      tic_tac_toe.display()
    check = tic_tac_toe.check_end_game()
    if (check != -1.0):
      return check
    i += 1

In [ ]:
# This function runs multiple games between two agents, agent_0 and agent_1
def run_multiple_games(num_games, agent_0, agent_1, disp):
  results = {"Agent 0 Wins": 0, "Agent 1 Wins": 0, "Draws": 0}
  for i in range(num_games):
    result = run_game(agent_0, agent_1, disp[i])
    print(f"Game {i+1}: Result: {result}")
    if (result == 0):
      results["Agent 0 Wins"] += 1
    if (result == 1):
      results["Agent 1 Wins"] += 1
    if (result == 2):
      results["Draws"] += 1
  return results

In [ ]:
# Create two more untrained agents
agent_2 = TicTacToeAgent(0)
agent_3 = TicTacToeAgent(1)

In [ ]:
# Play 1000 games each time and display only the first and last games
num_games = 1000
disp = [True] * 1 + [False] * (num_games - 2)  + [True] * 1

In [ ]:
# Run 1000 games between two untrained agents
results = run_multiple_games(num_games, agent_2, agent_3, disp)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|x|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|x|-|
|-|-|-|
|-|o|-|
‾‾‾‾‾‾‾
_______
|o|x|-|
|x|-|-|
|-|o|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|x|-|-|
|-|o|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|x|-|-|
|x|o|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|x|o|-|
|x|o|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|x|o|x|
|x|o|-|
‾‾‾‾‾‾‾
_______
|o|x|o|
|x|o|x|
|x|o|o|
‾‾‾‾‾‾‾
Game 1: Result: 0.0
Game 2: Result: 0.0
Game 3: Result: 0.0
Game 4: Result: 0.0
Game 5: Result: 1.0
Game 6: Result: 0.0
Game 7: Result: 0.0
Game 8: Result: 0.0
Game 9: Result: 1.0
Game 10: Result: 0.0
Game 11: Result: 0.0
Game 12: Result: 0.0
Game 13: Result: 1.0
Game 14: Result: 2.0
Game 15: Result: 1.0
Game 16: Result: 0.0
Game 17: Result: 0.0
Game 18: Result: 0.0
Game 19: Result: 1.0
Game 20: Result: 0.0
Game 21: Result: 0.0
Game 22: Result: 0.0
Game 23: Result: 1.0
Game 24: Result: 1.0
Game 25: Result: 0.0
Game 26: Result: 0.0
Game 27: Result: 2.0
Game 28: Result: 0.0
Game 29: Result: 2.0


In [ ]:
# The results are as follows:
results

{'Agent 0 Wins': 619, 'Agent 1 Wins': 255, 'Draws': 126}

In [ ]:
# Run 1000 games between the two trained agents
results = run_multiple_games(num_games, agent_0, agent_1, disp)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|x|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|o|o|-|
|x|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|o|o|x|
|x|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|o|o|x|
|x|-|o|
‾‾‾‾‾‾‾
_______
|x|-|-|
|o|o|x|
|x|-|o|
‾‾‾‾‾‾‾
_______
|x|-|-|
|o|o|x|
|x|o|o|
‾‾‾‾‾‾‾
_______
|x|x|-|
|o|o|x|
|x|o|o|
‾‾‾‾‾‾‾
_______
|x|x|o|
|o|o|x|
|x|o|o|
‾‾‾‾‾‾‾
Game 1: Result: 2.0
Game 2: Result: 2.0
Game 3: Result: 2.0
Game 4: Result: 2.0
Game 5: Result: 2.0
Game 6: Result: 2.0
Game 7: Result: 2.0
Game 8: Result: 2.0
Game 9: Result: 2.0
Game 10: Result: 2.0
Game 11: Result: 2.0
Game 12: Result: 2.0
Game 13: Result: 2.0
Game 14: Result: 2.0
Game 15: Result: 2.0
Game 16: Result: 2.0
Game 17: Result: 2.0
Game 18: Result: 2.0
Game 19: Result: 2.0
Game 20: Result: 2.0
Game 21: Result: 2.0
Game 22: Result: 2.0
Game 23: Result: 2.0
Game 24: Result: 2.0
Game 25: Result: 2.0
Game 26: Result: 2.0
Game 27: Result: 2.0
Game 28: Result: 2.0
Game 29: Result: 2.0


In [ ]:
# Every game is the same and a draw
results

{'Agent 0 Wins': 0, 'Agent 1 Wins': 0, 'Draws': 1000}

In [ ]:
# Run 1000 games between a trained 'o' agent and an untrained 'x' agent
results = run_multiple_games(num_games, agent_0, agent_3, disp)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|-|o|-|
|x|-|-|
‾‾‾‾‾‾‾
_______
|-|-|-|
|o|o|-|
|x|-|-|
‾‾‾‾‾‾‾
_______
|x|-|-|
|o|o|-|
|x|-|-|
‾‾‾‾‾‾‾
_______
|x|-|-|
|o|o|o|
|x|-|-|
‾‾‾‾‾‾‾
Game 1: Result: 0.0
Game 2: Result: 0.0
Game 3: Result: 0.0
Game 4: Result: 0.0
Game 5: Result: 0.0
Game 6: Result: 0.0
Game 7: Result: 0.0
Game 8: Result: 0.0
Game 9: Result: 0.0
Game 10: Result: 0.0
Game 11: Result: 0.0
Game 12: Result: 0.0
Game 13: Result: 0.0
Game 14: Result: 0.0
Game 15: Result: 0.0
Game 16: Result: 0.0
Game 17: Result: 0.0
Game 18: Result: 0.0
Game 19: Result: 0.0
Game 20: Result: 0.0
Game 21: Result: 0.0
Game 22: Result: 0.0
Game 23: Result: 0.0
Game 24: Result: 0.0
Game 25: Result: 0.0
Game 26: Result: 0.0
Game 27: Result: 0.0
Game 28: Result: 0.0
Game 29: Result: 0.0
Game 30: Result: 0.0
Game 31: Result: 0.0
Game 32: Result: 0.0
Game 33: Result: 0.0
Game 34: Result: 0.0
Game 35: Result: 0.0
Game 36: Result: 0.0
Game 37: Resu

In [ ]:
# The trained 'o' agent wins 991 games, while the untrained agents wins 0. There are 9 draws
results

{'Agent 0 Wins': 991, 'Agent 1 Wins': 0, 'Draws': 9}

In [ ]:
# Run 1000 games between an untrained 'o' agent and a trained 'x' agent
results = run_multiple_games(num_games, agent_2, agent_1, disp)

_______
|-|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|-|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|-|-|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|o|-|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|o|x|
|-|x|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|o|x|
|o|x|-|
|-|-|-|
‾‾‾‾‾‾‾
_______
|o|o|x|
|o|x|-|
|x|-|-|
‾‾‾‾‾‾‾
Game 1: Result: 1.0
Game 2: Result: 1.0
Game 3: Result: 1.0
Game 4: Result: 1.0
Game 5: Result: 1.0
Game 6: Result: 1.0
Game 7: Result: 2.0
Game 8: Result: 1.0
Game 9: Result: 1.0
Game 10: Result: 1.0
Game 11: Result: 1.0
Game 12: Result: 1.0
Game 13: Result: 1.0
Game 14: Result: 2.0
Game 15: Result: 1.0
Game 16: Result: 2.0
Game 17: Result: 1.0
Game 18: Result: 1.0
Game 19: Result: 1.0
Game 20: Result: 2.0
Game 21: Result: 1.0
Game 22: Result: 2.0
Game 23: Result: 1.0
Game 24: Result: 1.0
Game 25: Result: 1.0
Game 26: Result: 1.0
Game 27: Result: 1.0
Game 28: Result: 1.0
Game 29: Result: 1.0
Game 30: Result: 1.0
Game 31: Result: 2.0
Game 32: Result: 1.0
Game 33: Result: 1.0
Game 34: Result: 1.0
Game 35: Result

In [ ]:
# We see here that our trained 'x' agent has some flaws and loses 13 times. Maybe with a better formulation of the rewards or more training, the 'x' agent will learn to never lose.
results

{'Agent 0 Wins': 13, 'Agent 1 Wins': 910, 'Draws': 77}